In [1]:
from google.colab import drive
import pickle

# 📌 Montar Google Drive
drive.mount('/content/drive')

# 📌 Ruta del archivo guardado
ruta_carga = "/content/drive/MyDrive/Datos/1_ArchivosLeidos.pkl"


# 📌 Cargar el diccionario de DataFrames desde el archivo .pkl
with open(ruta_carga, "rb") as f:
    dataframes = pickle.load(f)

# 📌 Confirmación
print(f"\n✅ DataFrames cargados exitosamente desde: {ruta_carga}")
print(f"📊 Total de archivos en el diccionario: {len(dataframes)}")


# 📌 Verificar en qué DataFrames falta la columna 'NIT' antes de la corrección
initial_missing_nit_keys = [key for key, df in dataframes.items() if 'NIT' not in df.columns]

# 📢 Mostrar las bases donde no estaba estandarizado el nombre
if initial_missing_nit_keys:
    print("⚠️ Advertencia: Antes de la corrección, la columna 'NIT' no estaba presente en las siguientes claves:")
    for key in initial_missing_nit_keys:
        print(f"   ❌ {key}")
else:
    print("✅ Todas las bases ya tenían la columna 'NIT' correctamente nombrada.")

# 📌 Estandarizar el nombre de la columna "Nit" a "NIT" en todos los DataFrames
for key, df in dataframes.items():
    if 'Nit' in df.columns:
        df.rename(columns={'Nit': 'NIT'}, inplace=True)

# 📌 Verificar en qué DataFrames sigue faltando la columna 'NIT' después del cambio
final_missing_nit_keys = [key for key, df in dataframes.items() if 'NIT' not in df.columns]

# 📢 Mostrar el resultado final
if final_missing_nit_keys:
    print("⚠️ Advertencia: La columna 'NIT' sigue sin estar presente en las siguientes claves después del cambio:")
    for key in final_missing_nit_keys:
        print(f"   ❌ {key}")
else:
    print("✅ La columna 'NIT' ahora está presente en todas las bases de datos.")


from termcolor import colored  # Para colores en la terminal (opcional en Colab)

# 📌 Función para verificar consistencia de NITs en cada periodo
def verificar_consistencia_nits(years, keys):
    inconsistent_years = []

    for year in years:
        nit_sets = []
        valid_keys = [key.format(year) for key in keys if key.format(year) in dataframes]

        if len(valid_keys) == len(keys):
            for key in valid_keys:
                nit_sets.append(set(dataframes[key]['NIT']))

            if not all(nit_sets[0] == s for s in nit_sets):
                inconsistent_years.append(year)

    return inconsistent_years

# 📌 **1️⃣ Verificación Inicial**
print("\n🔎 **Verificación Inicial de Consistencia de NITs**")

inconsistencias_p1 = verificar_consistencia_nits(list(range(1995, 2004)) + [2007], ['{}_BG_ER', '{}_FE'])
inconsistencias_p2 = verificar_consistencia_nits(list(range(2004, 2007)) + list(range(2008, 2016)), ['{}_BG', '{}_FE', '{}_ER'])
inconsistencias_p3_py = verificar_consistencia_nits(list(range(2016, 2024)), ['{}_PY_CA', '{}_PY_FE', '{}_PY_ORI', '{}_PY_ER', '{}_PY_SF'])
inconsistencias_p3_pl = verificar_consistencia_nits(list(range(2016, 2024)), ['{}_PL_CA', '{}_PL_FE', '{}_PL_ORI', '{}_PL_ER', '{}_PL_SF'])

# 📢 Mostrar resultados iniciales
if inconsistencias_p1:
    print(colored(f"⚠️ Inconsistencias encontradas en el **Periodo 1**: {inconsistencias_p1}", "red"))
else:
    print(colored("✅ Todos los NITs son consistentes en el **Periodo 1**.", "green"))

if inconsistencias_p2:
    print(colored(f"⚠️ Inconsistencias encontradas en el **Periodo 2**: {inconsistencias_p2}", "red"))
else:
    print(colored("✅ Todos los NITs son consistentes en el **Periodo 2**.", "green"))

if inconsistencias_p3_py:
    print(colored(f"⚠️ Inconsistencias en las PYMES en el **Periodo 3**: {inconsistencias_p3_py}", "red"))
else:
    print(colored("✅ Todos los NITs en PYMES son consistentes en el **Periodo 3**.", "green"))

if inconsistencias_p3_pl:
    print(colored(f"⚠️ Inconsistencias en las PLENAS en el **Periodo 3**: {inconsistencias_p3_pl}", "red"))
else:
    print(colored("✅ Todos los NITs en PLENAS son consistentes en el **Periodo 3**.", "green"))


# 📌 **2️⃣ Corrección de inconsistencias en 2001**
if 2001 in inconsistencias_p1:
    print("\n⚠️ **Corrigiendo inconsistencias en 2001...**")

    # 📌 Obtener los conjuntos de NIT para 2001_BG_ER y 2001_FE
    nit_2001_BG_ER = set(dataframes['2001_BG_ER']['NIT'])
    nit_2001_FE = set(dataframes['2001_FE']['NIT'])

    # 📌 Detectar las discrepancias
    nit_discrepancia_BG_ER = nit_2001_BG_ER - nit_2001_FE
    nit_discrepancia_FE = nit_2001_FE - nit_2001_BG_ER

    # 📌 Identificar las filas que contienen los NITs en conflicto
    filas_discrepancia_BG_ER = dataframes['2001_BG_ER'][dataframes['2001_BG_ER']['NIT'].isin(nit_discrepancia_BG_ER)].index
    filas_discrepancia_FE = dataframes['2001_FE'][dataframes['2001_FE']['NIT'].isin(nit_discrepancia_FE)].index

    # 📢 Mostrar los índices de las filas a eliminar
    print("📌 Fila(s) en 2001_BG_ER a eliminar:", list(filas_discrepancia_BG_ER))
    print("📌 Fila(s) en 2001_FE a eliminar:", list(filas_discrepancia_FE))

    # 📌 Eliminar las filas con discrepancias
    dataframes['2001_BG_ER'].drop(filas_discrepancia_BG_ER, inplace=True)
    dataframes['2001_FE'].drop(filas_discrepancia_FE, inplace=True)

    print("✅ Se han eliminado las filas con discrepancias en los NITs de 2001.")


# 📌 **3️⃣ Verificación Final**
print("\n🔎 **Verificación Final de Consistencia de NITs**")

inconsistencias_p1 = verificar_consistencia_nits(list(range(1995, 2004)) + [2007], ['{}_BG_ER', '{}_FE'])
inconsistencias_p2 = verificar_consistencia_nits(list(range(2004, 2007)) + list(range(2008, 2016)), ['{}_BG', '{}_FE', '{}_ER'])
inconsistencias_p3_py = verificar_consistencia_nits(list(range(2016, 2024)), ['{}_PY_CA', '{}_PY_FE', '{}_PY_ORI', '{}_PY_ER', '{}_PY_SF'])
inconsistencias_p3_pl = verificar_consistencia_nits(list(range(2016, 2024)), ['{}_PL_CA', '{}_PL_FE', '{}_PL_ORI', '{}_PL_ER', '{}_PL_SF'])

# 📢 Mostrar resultados finales
if inconsistencias_p1:
    print(colored(f"⚠️ Inconsistencias encontradas en el **Periodo 1** después de la corrección: {inconsistencias_p1}", "red"))
else:
    print(colored("✅ Todos los NITs son consistentes en el **Periodo 1** después de la corrección.", "green"))

if inconsistencias_p2:
    print(colored(f"⚠️ Inconsistencias encontradas en el **Periodo 2** después de la corrección: {inconsistencias_p2}", "red"))
else:
    print(colored("✅ Todos los NITs son consistentes en el **Periodo 2** después de la corrección.", "green"))

if inconsistencias_p3_py:
    print(colored(f"⚠️ Inconsistencias en las PYMES en el **Periodo 3** después de la corrección: {inconsistencias_p3_py}", "red"))
else:
    print(colored("✅ Todos los NITs en PYMES son consistentes en el **Periodo 3** después de la corrección.", "green"))

if inconsistencias_p3_pl:
    print(colored(f"⚠️ Inconsistencias en las PLENAS en el **Periodo 3** después de la corrección: {inconsistencias_p3_pl}", "red"))
else:
    print(colored("✅ Todos los NITs en PLENAS son consistentes en el **Periodo 3** después de la corrección.", "green"))


# 📌 Creación de la base inicial con columnas NIT y AÑO
import pandas as pd

# Crear DataFrame vacío para la BaseInicial
base_inicial = pd.DataFrame(columns=['NIT', 'Año'])

# 📌 Función para extraer NIT y Año de cada base de datos y añadirlos a base_inicial
def agregar_a_base_inicial(years, key_template):
    global base_inicial  # Para modificar la variable fuera de la función
    for year in years:
        key = key_template.format(year)

        if key in dataframes:
            df = dataframes[key]

            # Extraer valores de NIT y Año
            nit_values = df['NIT'].values
            year_values = [year] * len(nit_values)

            # Agregar al DataFrame base_inicial
            base_inicial = pd.concat([base_inicial, pd.DataFrame({'NIT': nit_values, 'Año': year_values})], ignore_index=True)
        else:
            print(f"⚠️ No se encontraron datos para {key}.")

# 📌 Periodo 1: Años 1995-2003 y 2007 (_BG_ER)
agregar_a_base_inicial(list(range(1995, 2004)) + [2007], "{}_BG_ER")

# 📌 Periodo 2: Años 2004-2015 (excepto 2007) (_BG)
agregar_a_base_inicial(list(range(2004, 2007)) + list(range(2008, 2016)), "{}_BG")

# 📌 Periodo 3: Años 2016-2022 (_PY_CA y _PL_CA)
for year in range(2016, 2024):
    for key_template in ["{}_PY_CA", "{}_PL_CA"]:
        agregar_a_base_inicial([year], key_template)

# 📌 Mostrar resultados
print("\n🔎 **Vista previa de la Base Inicial creada**")
print(base_inicial.head())
print(base_inicial.tail())
print(f"\n✅ Total de registros en `base_inicial`: {len(base_inicial)}")



import difflib
from tqdm import tqdm  # Barra de progreso

# Definir los sufijos para cada periodo
period_1_suffixes = ['_BG_ER']
period_2_suffixes = ['_BG']
period_3_suffixes = ['_PY_CA', '_PL_CA']

# Definir los años por periodo
years_period_1 = list(range(1995, 2004)) + [2007]
years_period_2 = list(range(2004, 2007)) + list(range(2008, 2016))
years_period_3 = list(range(2016, 2024))

# Definir las columnas a buscar para cada variable
columns_to_search = {
    'Ciudad': ['Ciudad', 'CIUDAD', 'Ciudad de la dirección del domicilio'],
    'CIIU': ['CIIU', 'Clasificación Industrial Internacional Uniforme Versión 4 A.C', 'Clasificación Industrial Internacional Uniforme Versión 4 A.C (CIIU)'],
    'DEP': ['Dpto','DPTO', 'DEPTO', 'Departamento', 'DEPARTAMENTO', 'Departamento de la dirección del domicilio']  # 📌 Cambio de 'Dpto' a 'DEP'
}

# Cambiar nombres de columnas en todas las bases de datos
for variable, nombres_columnas in columns_to_search.items():
    for key, dataframe in dataframes.items():
        for column_to_search in nombres_columnas:
            if column_to_search in dataframe.columns:
                dataframe.rename(columns={column_to_search: variable}, inplace=True)

# Verificar que las columnas existen en todas las bases de datos
for column in ['Ciudad', 'CIIU', 'DEP']:  # 📌 Incluido 'DEP'
    for suffixes, years in zip([period_1_suffixes, period_2_suffixes, period_3_suffixes], [years_period_1, years_period_2, years_period_3]):
        for suffix in suffixes:
            for year in years:
                key = f'{year}{suffix}'
                if key in dataframes and column not in dataframes[key].columns:
                    print(f"⚠️ La columna '{column}' no se encontró en '{key}'")

# Crear las columnas CI, CIIU y DEP en base_inicial
base_inicial['Ciudad'] = None
base_inicial['CIIU'] = None
base_inicial['DEP'] = None  # 📌 Cambio de 'Dpto' a 'DEP'

# Iterar sobre cada fila de base_inicial con barra de progreso
print("\n🔄 **Procesando datos...**")
for index, row in tqdm(base_inicial.iterrows(), total=len(base_inicial), desc="Progreso"):

    nit = row['NIT']
    year = row['Año']

    # Determinar sufijo
    if 1995 <= year <= 2003 or year == 2007:
        suffix = '_BG_ER'
    elif 2004 <= year <= 2015 and year != 2007:
        suffix = '_BG'
    elif 2016 <= year <= 2023:
        suffix = '_PY_CA'
        db_name = f"{year}{suffix}"
        if db_name in dataframes and nit in dataframes[db_name]['NIT'].values:
            suffix = '_PY_CA'
        else:
            suffix = '_PL_CA'
    else:
        continue  # Año inválido, no hacer nada

    # Crear nombre de la base de datos
    db_name = f"{year}{suffix}"

    # Verificar si la base de datos existe
    if db_name in dataframes:
        df = dataframes[db_name]
        match = df[df['NIT'] == nit]
        if not match.empty:
            base_inicial.at[index, 'Ciudad'] = match['Ciudad'].values[0]
            base_inicial.at[index, 'CIIU'] = match['CIIU'].values[0]

            # 📌 Agregar el Departamento (ahora llamado "DEP")
            if 'DEP' in match.columns:
                base_inicial.at[index, 'DEP'] = match['DEP'].values[0]

# ✅ **Verificación final**
print("\n📌 **Verificando que todas las filas tengan valores en Ciudad, CIIU y DEP:**")
missing_ci = base_inicial['Ciudad'].isna().sum()
missing_ciiu = base_inicial['CIIU'].isna().sum()
missing_dep = base_inicial['DEP'].isna().sum()
print(f"🔍 Faltan {missing_ci} valores en 'Ciudad'")
print(f"🔍 Faltan {missing_ciiu} valores en 'CIIU'")
print(f"🔍 Faltan {missing_dep} valores en 'DEP'")  # 📌 Verificación adicional

# ✅ **Mostrar resultados finales**
from IPython.display import display
print("\n📌 **Base Inicial después de agregar Ciudad, CIIU y DEP:**")
display(base_inicial)

print("✅ ¡Proceso completado!")



import difflib
from tqdm import tqdm  # Barra de progreso

# Definir los sufijos para cada periodo
period_1_suffixes = ['_BG_ER']
period_2_suffixes = ['_BG']
period_3_suffixes = ['_PY_SF', '_PL_SF']

# Definir los años por periodo
years_period_1 = list(range(1995, 2004)) + [2007]
years_period_2 = list(range(2004, 2007)) + list(range(2008, 2016))
years_period_3 = list(range(2016, 2024))

# Definir las cuentas a buscar para cada variable
accounts_to_search = {
    'AC': ['0 TOTAL ACTIVO CORRIENTE','TOTAL ACTIVO CORRIENTE  ','TOTAL ACTIVO CORRIENTE','_TOTAL_ACTIVO_CORRIENTE','Activos corrientes totales (CurrentAssets)'],
    'I': ['14 SUBTOTAL INVENTARIOS','14 SUBTOTAL INVENTARIOS  (CP)','14 SUBTOTAL INVENTARIOS (CP)','14 SUBTOTAL INVENTARIOS  (CP)','_14_SUBTOTAL_INVENTARIOS_CP_','Inventarios corrientes (Inventories)'],
    'AF': ['15 PROPIEDADES PLANTA Y EQUIPO NETO               ','15 PROPIEDADES PLANTA Y EQUIPO(ANEXO 9) ','15 PROPIEDADES PLANTA Y EQUIPO','15 PROPIEDADES PLANTA Y EQUIPO(ANEXO 9)','_15_PROPIEDADES_PLANTA_Y_EQUIPO','Propiedades, planta y equipo (PropertyPlantAndEquipment)'],
    'AT': ['0 TOTAL ACTIVO                                    ','TOTAL ACTIVO  ','TOTAL ACTIVO','_TOTAL_ACTIVO','Total de activos (Assets)'],
    'PN': ['0 TOTAL PATRIMONIO                                ','TOTAL PATRIMONIO','_TOTAL_PATRIMONIO','Patrimonio total (Equity)'],
    'PLP': ['0 TOTAL PASIVO NO CORRIENTE                       ','TOTAL PASIVO NO CORRIENTE ','TOTAL PASIVO NO CORRIENTE','_TOTAL_PASIVO_NO_CORRIENTE','Total de pasivos no corrientes (NoncurrentLiabilities)'],
    'PCP': ['0 TOTAL PASIVO CORRIENTE','TOTAL PASIVO CORRIENTE ','TOTAL PASIVO CORRIENTE','_TOTAL_PASIVO_CORRIENTE_','Pasivos corrientes totales (CurrentLiabilities)' ],
    'PT': ['0 TOTAL PASIVO                                    ','TOTAL PASIVO','_TOTAL_PASIVO','Total pasivos (Liabilities)'],
    'CPC': ['13 SUBTOTAL DEUDORES CORTO PLAZO','13 SUBTOTAL DEUDORES (CP)','13 SUBTOTAL DEUDORES (CP)','_13_SUBTOTAL_DEUDORES_CP_','Cuentas comerciales por cobrar y otras cuentas por cobrar no corrientes (NoncurrentReceivables)'],
    'CPP': ['23 SUBTOTAL CUENTAS POR PAGAR CORTO PLAZO','23 SUBTOTAL CUENTAS POR PAGAR (CP)','23 SUBTOTAL CUENTAS POR PAGAR (CP)','_23_SUBTOTAL_CUENTAS_POR_PAGAR_CP_','Cuentas comerciales por pagar y otras cuentas por pagar no corrientes (NoncurrentPayables)']
}

# Cambiar nombres de columnas en todas las bases de datos
for variable, nombres_columnas in accounts_to_search.items():
    for key, dataframe in dataframes.items():
        for column_to_search in nombres_columnas:
            if column_to_search in dataframe.columns:
                dataframe.rename(columns={column_to_search: variable}, inplace=True)

# Verificar que las columnas existen en todas las bases de datos
for column in accounts_to_search.keys():
    for suffixes, years in zip([period_1_suffixes, period_2_suffixes, period_3_suffixes], [years_period_1, years_period_2, years_period_3]):
        for suffix in suffixes:
            for year in years:
                key = f'{year}{suffix}'
                if key in dataframes and column not in dataframes[key].columns:
                    print(f"⚠️ La columna '{column}' no se encontró en '{key}'")

# Crear las nuevas columnas en base_inicial
for variable in accounts_to_search.keys():
    base_inicial[variable] = None

# Iterar sobre cada fila de base_inicial con barra de progreso
print("\n🔄 **Procesando datos del activo...**")
for index, row in tqdm(base_inicial.iterrows(), total=len(base_inicial), desc="Progreso"):

    nit = row['NIT']
    year = row['Año']

    # Determinar sufijo
    if 1995 <= year <= 2003 or year == 2007:
        suffix = '_BG_ER'
    elif 2004 <= year <= 2015 and year != 2007:
        suffix = '_BG'
    elif 2016 <= year <= 2023:
        suffix = '_PY_SF'
        db_name = f"{year}{suffix}"
        if db_name in dataframes and nit in dataframes[db_name]['NIT'].values:
            suffix = '_PY_SF'
        else:
            suffix = '_PL_SF'
    else:
        continue  # Año inválido, no hacer nada

    # Crear nombre de la base de datos
    db_name = f"{year}{suffix}"

    # Verificar si la base de datos existe
    if db_name in dataframes:
        df = dataframes[db_name]
        match = df[df['NIT'] == nit]
        if not match.empty:
            for column in accounts_to_search.keys():
                base_inicial.at[index, column] = match[column].values[0] if column in match.columns else None

# ✅ **Verificación final**
print("\n📌 **Verificando que todas las filas tengan valores en las cuentas del activo:**")
for column in accounts_to_search.keys():
    missing_values = base_inicial[column].isna().sum()
    print(f"🔍 Faltan {missing_values} valores en '{column}'")

# ✅ **Mostrar resultados finales**
from IPython.display import display
print("\n📌 **Base Inicial después de agregar cuentas del activo:**")
display(base_inicial)

print("✅ ¡Proceso completado!")


import difflib
from tqdm import tqdm  # Barra de progreso

# Definir los sufijos para cada periodo
period_1_suffixes = ['_BG_ER']
period_2_suffixes = ['_ER']
period_3_suffixes = ['_PY_ER', '_PL_ER']

# Definir los años por periodo
years_period_1 = list(range(1995, 2004)) + [2007]
years_period_2 = list(range(2004, 2007)) + list(range(2008, 2016))
years_period_3 = list(range(2016, 2024))

# Definir las cuentas a buscar para cada variable
accounts_to_search = {
    'UN': ['59 GANANCIAS Y PERDIDAS                           ','59 GANANCIAS Y PERDIDAS','_59_GANANCIAS_Y_PERDIDAS','Ganancia (pérdida) (ProfitLoss)'],
    'V': ['41 INGRESOS OPERACIONALES                         ','41 INGRESOS OPERACIONALES (ANEXO 1)','41 INGRESOS OPERACIONALES ','41 INGRESOS OPERACIONALES (ANEXO  1)','_41_INGRESOS_OPERACIONALES','Ingresos de actividades ordinarias (Revenue)'],
    'UB': ['0 UTILIDAD BRUTA                                  ','UTILIDAD BRUTA','UTILIDAD_BRUTA','Ganancia bruta (GrossProfit)'],
    'EBIT': ['0 UTILIDAD NETA ANTES DE IMPUESTOS                ','UTILIDAD ANTES DE IMPUESTOS Y AJUSTES POR INFLACION','UTILIDAD ANTES DE IMPUESTOS','UTILIDAD_ANTES_DE_IMPUESTOS','Ganancia (pérdida), antes de impuestos (ProfitLossBeforeTax)'],
    'UO': ['0 UTILIDAD OPERACIONAL                            ','UTILIDAD OPERACIONAL','UTILIDAD_OPERACIONAL','Ganancia (pérdida) por actividades de operación (ProfitLossFromOperatingActivities)','Ganancia (pérdida) por actividades de operación (GananciaPerdidaPorActividadesDeOperacion)'],
    'GF': ['53 MENOS GASTOS NO OPERACIONALES                  ','53 MENOS: GASTOS NO OPERACIONALES (ANEXO 5)','53 MENOS: GASTOS NO OPERACIONALES ','53 MENOS: GASTOS NO OPERACIONALES  (ANEXO  5)','_53_MENOS_GASTOS_NO_OPERACIONALES','Costos financieros (FinanceCosts)'],
    'GV': ['52 MENOS GASTOS OPERACIONALES DE VENTAS           ','52 MENOS: GASTOS OPERACIONALES DE VENTAS (ANEXO 3)','52 MENOS: GASTOS OPERACIONALES DE VENTAS ','52 MENOS: GASTOS OPERACIONALES DE VENTAS  (ANEXO  3)','_52_MENOS_GASTOS_OPERACIONALES_DE_VENTAS','52 MENOS: GASTOS OPERACIONALES DE VENTAS ','Gastos de ventas (DistributionCosts)'],
    'GA': ['51 MENOS GASTOS OPERACIONALES ADMON.              ','51 MENOS GASTOS OPERACIONALES ADMON.              ','51 MENOS: GASTOS OPERACIONALES DE ADMINISTRACIÓN (ANEXO 3)','51 MENOS: GASTOS OPERACIONALES DE ADMINISTRACIÓN ','51 MENOS: GASTOS OPERACIONALES DE  ADMINISTRACIÓN (ANEXO  3)','_51_MENOS_GASTOS_OPERACIONALES_DE__ADMINISTRACION','51 MENOS: GASTOS OPERACIONALES DE ADMINISTRACIÓN ','51 MENOS: GASTOS OPERACIONALES DE ADMINISTRACIÃ“N ','Gastos de administración (AdministrativeExpense)'],
    'Imp': ['54 MENOS IMPUESTOS DE RENTA Y COMPLEMENTARIOS     ','54 MENOS: IMPUESTO DE RENTA Y COMPLEMENTARIOS','_54_MENOS_IMPUESTO_DE_RENTA_Y_COMPLEMENTARIOS','Ingreso (gasto) por impuestos (IncomeTaxExpenseContinuingOperations)']
}

# Cambiar nombres de columnas en todas las bases de datos
for variable, nombres_columnas in accounts_to_search.items():
    for key, dataframe in dataframes.items():
        for column_to_search in nombres_columnas:
            if column_to_search in dataframe.columns:
                dataframe.rename(columns={column_to_search: variable}, inplace=True)

# Verificar que las columnas existen en todas las bases de datos
for column in accounts_to_search.keys():
    for suffixes, years in zip([period_1_suffixes, period_2_suffixes, period_3_suffixes], [years_period_1, years_period_2, years_period_3]):
        for suffix in suffixes:
            for year in years:
                key = f'{year}{suffix}'
                if key in dataframes and column not in dataframes[key].columns:
                    print(f"⚠️ La columna '{column}' no se encontró en '{key}'")

# Crear las nuevas columnas en base_inicial
for variable in accounts_to_search.keys():
    base_inicial[variable] = None

# Iterar sobre cada fila de base_inicial con barra de progreso
print("\n🔄 **Procesando datos de ingresos y utilidades...**")
for index, row in tqdm(base_inicial.iterrows(), total=len(base_inicial), desc="Progreso"):

    nit = row['NIT']
    year = row['Año']

    # Determinar sufijo
    if 1995 <= year <= 2003 or year == 2007:
        suffix = '_BG_ER'
    elif 2004 <= year <= 2015 and year != 2007:
        suffix = '_ER'
    elif 2016 <= year <= 2023:
        suffix = '_PY_ER'
        db_name = f"{year}{suffix}"
        if db_name in dataframes and nit in dataframes[db_name]['NIT'].values:
            suffix = '_PY_ER'
        else:
            suffix = '_PL_ER'
    else:
        continue  # Año inválido, no hacer nada

    # Crear nombre de la base de datos
    db_name = f"{year}{suffix}"

    # Verificar si la base de datos existe
    if db_name in dataframes:
        df = dataframes[db_name]
        match = df[df['NIT'] == nit]
        if not match.empty:
            for column in accounts_to_search.keys():
                base_inicial.at[index, column] = match[column].values[0] if column in match.columns else None

# ✅ **Verificación final**
print("\n📌 **Verificando que todas las filas tengan valores en las cuentas de ingresos y utilidades:**")
for column in accounts_to_search.keys():
    missing_values = base_inicial[column].isna().sum()
    print(f"🔍 Faltan {missing_values} valores en '{column}'")

# ✅ **Mostrar resultados finales**
from IPython.display import display
print("\n📌 **Base Inicial después de agregar cuentas de ingresos y utilidades:**")
display(base_inicial)

print("✅ ¡Proceso completado!")

import difflib
from tqdm import tqdm  # Barra de progreso

# Definir los sufijos para cada periodo
period_1_suffixes = ['_FE']
period_2_suffixes = ['_FE']
period_3_suffixes = ['_PY_FE', '_PL_FE']

# Definir los años por periodo
years_period_1 = list(range(1995, 2004)) + [2007]
years_period_2 = list(range(2004, 2007)) + list(range(2008, 2016))
years_period_3 = list(range(2016, 2024))

# Definir las cuentas a buscar
accounts_to_search = {
    'DyA1': ['(+) Depreciacion (Anexo 3)','+ Depreciaciones','(+)   Depreciaciones (Anexo 3)                    ','(+) DEPRECIACIONES (ANEXO 3)','(+) DEPRECIACIONES ','(+)  DEPRECIACIONES (ANEXO 3)','MAS_DEPRECIACIONES','(+) DEPRECIACIONES '],
    'DyA2': ['(+) Amortizacion (Anexo 3)','+ Amortizaciones','(+)   Amortizaciones (Anexo 3)                    ','(+) AMORTIZACIONES (ANEXO 3)','(+) AMORTIZACIONES ','(+)  AMORTIZACIONES (ANEXO 3)','MAS_AMORTIZACIONES','(+) AMORTIZACIONES '],
    'DyA': ['( + ) Ajustes por gastos de depreciación y amortización (AdjustmentsForDepreciationAndAmortisationExpense)'],
    'FCL1': ['FLUJO_DE_EFECTIVO_NETO_EN_ACTIVIDADES_DE_OPERACION','Flujo de Efectivo Neto en Actividades de Operación','TOTAL 2 - Flujo de Efectivo Neto en Actividades de','Flujo de Efectivo neto en actividades de operación','FLUJO DE EFECTIVO NETO EN ACTIVIDADES DE OPERACIÓN','FLUJO DE EFECTIVO NETO EN ACTIVIDADES DE OPERACIÃ“N','Flujos de efectivo netos procedentes de (utilizados en) actividades de operación (CashFlowsFromUsedInOperatingActivities)'],
    'FCL2': ['FLUJO_DE_EFECTIVO_NETO_EN_ACTIVIDADES_DE_INVERSION','Flujo de Efectivo Neto en Actividades de Inversion','TOTAL 3 - Flujo de Efectivo Neto en Actividades de','Flujo de Efectivo neto en activ. de inversión(R.21','Flujo de Efectivo Neto en Actividades de Inversion','FLUJO DE EFECTIVO NETO EN ACTIVIDADES DE INVERSION','FLUJO DE EFECTIVO NETO EN ACTIVIDADES DE INVERSION','Flujos de efectivo netos procedentes de (utilizados en) actividades de inversión (CashFlowsFromUsedInInvestingActivities)','FLUJO DE EFECTIVO NETO EN ACTIVIDADES DE INVERSION']
}

# Renombrar columnas en todas las bases de datos
for variable, nombres_columnas in accounts_to_search.items():
    for key, dataframe in dataframes.items():
        for column_to_search in nombres_columnas:
            if column_to_search in dataframe.columns:
                dataframe.rename(columns={column_to_search: variable}, inplace=True)

# 🔍 **Verificación de columnas en los periodos correctos**
for column, valid_years in [('DyA1', years_period_1 + years_period_2),
                            ('DyA2', years_period_1 + years_period_2),
                            ('DyA', years_period_3),
                            ('FCL1', years_period_1 + years_period_2 + years_period_3),
                            ('FCL2', years_period_1 + years_period_2 + years_period_3)]:
    for suffixes, years in zip([period_1_suffixes, period_2_suffixes, period_3_suffixes], [years_period_1, years_period_2, years_period_3]):
        for suffix in suffixes:
            for year in years:
                if year in valid_years:  # Solo busca en los años correctos
                    key = f'{year}{suffix}'
                    if key in dataframes and column not in dataframes[key].columns:
                        print(f"⚠️ La columna '{column}' no se encontró en '{key}'")

# Crear columnas en base_inicial
base_inicial['DyA'] = None
base_inicial['FCL'] = None

# Iterar sobre cada fila con barra de progreso
print("\n🔄 **Procesando Depreciación, Amortización y Flujo de Efectivo...**")
for index, row in tqdm(base_inicial.iterrows(), total=len(base_inicial), desc="Progreso"):

    nit = row['NIT']
    year = row['Año']

    # Determinar sufijo correspondiente
    if 1995 <= year <= 2003 or year == 2007:
        suffix = '_FE'
    elif 2004 <= year <= 2015 and year != 2007:
        suffix = '_FE'
    elif 2016 <= year <= 2023:
        suffix = '_PY_FE'
        db_name = f"{year}{suffix}"
        if db_name in dataframes and nit in dataframes[db_name]['NIT'].values:
            suffix = '_PY_FE'
        else:
            suffix = '_PL_FE'
    else:
        continue  # Año inválido, no hacer nada

    db_name = f"{year}{suffix}"

    # Verificar si la base de datos existe
    if db_name in dataframes:
        df = dataframes[db_name]
        match = df[df['NIT'] == nit]
        if not match.empty:
            # Asignar DyA (Depreciación y Amortización)
            if year < 2016:  # Antes de 2016 se suman DyA1 y DyA2
                dyA1_value = match['DyA1'].values[0] if 'DyA1' in match.columns else 0
                dyA2_value = match['DyA2'].values[0] if 'DyA2' in match.columns else 0
                base_inicial.at[index, 'DyA'] = dyA1_value + dyA2_value if (dyA1_value and dyA2_value) else None
            else:  # Desde 2016 se usa directamente DyA
                dyA_value = match['DyA'].values[0] if 'DyA' in match.columns else None
                base_inicial.at[index, 'DyA'] = dyA_value

            # Asignar FCL (Flujo de caja libre)
            fcl1_value = match['FCL1'].values[0] if 'FCL1' in match.columns else 0
            fcl2_value = match['FCL2'].values[0] if 'FCL2' in match.columns else 0
            base_inicial.at[index, 'FCL'] = fcl1_value - fcl2_value if (fcl1_value and fcl2_value) else None

# ✅ **Verificación final**
print("\n📌 **Verificando que todas las filas tengan valores en DyA y FCL:**")
missing_dya = base_inicial['DyA'].isna().sum()
missing_fcl = base_inicial['FCL'].isna().sum()
print(f"🔍 Faltan {missing_dya} valores en 'DyA'")
print(f"🔍 Faltan {missing_fcl} valores en 'FCL'")

# ✅ **Mostrar resultados finales**
from IPython.display import display
print("\n📌 **Base Inicial después de agregar DyA y FCL:**")
display(base_inicial)

print("✅ ¡Proceso completado!")

import os
import pickle

# 📂 Ruta donde se guardará el archivo
ruta_base_inicial = "/content/drive/MyDrive/Datos/2_1_BaseInicial.pkl"


# 📌 Crear la carpeta si no existe
os.makedirs(os.path.dirname(ruta_base_inicial), exist_ok=True)

# 🧹 Eliminar duplicados por NIT y Año (conservando el primero que aparezca)
base_inicial.drop_duplicates(subset=['NIT', 'Año'], keep='first', inplace=True)
print(f"📦 Registros únicos por NIT y Año: {base_inicial.shape[0]}")


# 📌 Guardar en formato Pickle
base_inicial.to_pickle(ruta_base_inicial)

print(f"\n✅ `2_BaseInicial.pkl` guardado en: {ruta_base_inicial}")


# 📌 Ruta para guardar en formato Parquet
ruta_parquet = "/content/drive/MyDrive/Datos/2_2_BaseInicial.parquet"


# 📌 Guardar en formato Parquet
base_inicial.to_parquet(ruta_parquet, index=False)
print(f"✅ `2_BaseInicial.parquet` guardado en: {ruta_parquet}")

# 📌 RESUMEN FINAL DE LA BASE INICIAL
print("\n📊 RESUMEN DE LA BASE FINAL")

total_registros = base_inicial.shape[0]
total_nits = base_inicial['NIT'].nunique()
total_años = base_inicial['Año'].nunique()
columnas = list(base_inicial.columns)

print(f"✅ Total de registros únicos: {total_registros}")
print(f"👤 NITs únicos: {total_nits}")
print(f"📅 Años únicos: {total_años}")
print(f"📄 Total de columnas: {len(columnas)}")

# Mostrar primeras 10 NITs
print("\n🆔 Ejemplos de NITs:")
print(base_inicial['NIT'].dropna().astype(str).unique()[:10])

# Mostrar primeros 10 años ordenados
print("\n📅 Años presentes:")
print(sorted(base_inicial['Año'].dropna().unique())[:10])

# Mostrar nombres de columnas
print("\n📋 Columnas disponibles:")
for col in columnas:
    print(f"   • {col}")
# 📄 Crear resumen de paso en archivo .txt
resumen_txt = f"""
📄 Script: 2_Creacion_Base_Inicial.ipynb
📅 Fecha: 2025-05-16
🧩 Objetivo: Construir una base consolidada por empresa y año con información financiera estandarizada.

🔁 Pasos realizados:
- Carga desde: 1_ArchivosLeidos.pkl
- Estandarización de columnas: NIT, Ciudad, CIIU, DEP
- Corrección de inconsistencias por año en NITs
- Extracción de variables contables del activo, resultado y flujo
- Eliminación de duplicados por NIT y Año

📦 Archivos generados:
- 2_1_BaseInicial.pkl
- 2_2_BaseInicial.parquet

📊 Resumen de la base:
- Registros únicos (NIT-Año): {base_inicial.shape[0]}
- NITs únicos: {base_inicial['NIT'].nunique()}
- Años únicos: {base_inicial['Año'].nunique()}
- Columnas disponibles: {len(base_inicial.columns)}
"""

ruta_resumen = "/content/drive/MyDrive/Datos/2_resumen.txt"
with open(ruta_resumen, "w", encoding="utf-8") as f:
    f.write(resumen_txt.strip())

print(f"📝 Resumen guardado en: {ruta_resumen}")



Mounted at /content/drive

✅ DataFrames cargados exitosamente desde: /content/drive/MyDrive/Datos/1_ArchivosLeidos.pkl
📊 Total de archivos en el diccionario: 133
⚠️ Advertencia: Antes de la corrección, la columna 'NIT' no estaba presente en las siguientes claves:
   ❌ 2001_FE
   ❌ 2002_FE
   ❌ 2003_FE
✅ La columna 'NIT' ahora está presente en todas las bases de datos.

🔎 **Verificación Inicial de Consistencia de NITs**
⚠️ Inconsistencias encontradas en el **Periodo 1**: [2001]
✅ Todos los NITs son consistentes en el **Periodo 2**.
✅ Todos los NITs en PYMES son consistentes en el **Periodo 3**.
✅ Todos los NITs en PLENAS son consistentes en el **Periodo 3**.

⚠️ **Corrigiendo inconsistencias en 2001...**
📌 Fila(s) en 2001_BG_ER a eliminar: [6496]
📌 Fila(s) en 2001_FE a eliminar: []
✅ Se han eliminado las filas con discrepancias en los NITs de 2001.

🔎 **Verificación Final de Consistencia de NITs**
✅ Todos los NITs son consistentes en el **Periodo 1** después de la corrección.
✅ Todos lo

Progreso: 100%|██████████| 542084/542084 [05:53<00:00, 1532.83it/s]



📌 **Verificando que todas las filas tengan valores en Ciudad, CIIU y DEP:**
🔍 Faltan 87 valores en 'Ciudad'
🔍 Faltan 88 valores en 'CIIU'
🔍 Faltan 25490 valores en 'DEP'

📌 **Base Inicial después de agregar Ciudad, CIIU y DEP:**


,NIT,Año,Ciudad,CIIU,DEP
0,860067934,1995,BOGOTA D.C.,A0111,BOGOTA D.C.
1,890328003,1995,CALI,A0111,VALLE
2,891302216,1995,CALI,A0111,VALLE
3,800160353,1995,CALI,A0111,VALLE
4,800063990,1995,CALI,A0111,VALLE
...,...,...,...,...,...
542079,901656609,2023,MEDELLIN-ANTIOQUIA,J6201 - Actividades de desarrollo de sistemas ...,ANTIOQUIA
542080,901675255,2023,BOGOTA D.C.,N8291 - Actividades de agencias de cobranza y ...,BOGOTA D.C.
542081,901680258,2023,YARUMAL-ANTIOQUIA,C1040 - Elaboración de productos lácteos,ANTIOQUIA
542082,901712459,2023,BOGOTA D.C.,H4921 - Transporte de pasajeros,BOGOTA D.C.


✅ ¡Proceso completado!

🔄 **Procesando datos del activo...**


...

✅ ¡Proceso completado!

🔄 **Procesando Depreciación, Amortización y Flujo de Efectivo...**


Progreso: 100%|██████████| 542084/542084 [06:11<00:00, 1457.87it/s]



📌 **Verificando que todas las filas tengan valores en DyA y FCL:**
🔍 Faltan 281483 valores en 'DyA'
🔍 Faltan 126839 valores en 'FCL'

📌 **Base Inicial después de agregar DyA y FCL:**


,NIT,Año,Ciudad,CIIU,DEP,AC,I,AF,AT,PN,...,V,UB,EBIT,UO,GF,GV,GA,Imp,DyA,FCL
0,860067934,1995,BOGOTA D.C.,A0111,BOGOTA D.C.,239648,0,836854,1333751,664905,...,1031974,236113,-30835,161149,212770,0,74964,24625,186658,139410
1,890328003,1995,CALI,A0111,VALLE,682933,172732,363257,2955905,2881797,...,839772,405640,84292,105484,24555,0,300156,15967,None,135254
2,891302216,1995,CALI,A0111,VALLE,649461,380479,506443,1307972,845568,...,633557,297693,5446,163513,159567,0,134180,6916,None,-13312
3,800160353,1995,CALI,A0111,VALLE,768173,367162,1550320,2402577,2379993,...,335610,194706,35376,35222,19850,0,159484,24250,None,-200479
4,800063990,1995,CALI,A0111,VALLE,211551,17227,1123495,2149570,732193,...,441379,100847,-182689,-1749,269257,31916,70680,15121,None,175046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542079,901656609,2023,MEDELLIN-ANTIOQUIA,J6201 - Actividades de desarrollo de sistemas ...,ANTIOQUIA,1869704.0,NaN,436898.0,2754463.0,149723.0,...,5434127.0,5434127.0,-2898138,-2898138.0,NaN,NaN,8333838.0,-447861.0,174076.0,295509.0
542080,901675255,2023,BOGOTA D.C.,N8291 - Actividades de agencias de cobranza y ...,BOGOTA D.C.,26057893.0,NaN,NaN,26297890.0,889168.0,...,38956677.0,11233651.0,1399887,1481400.0,590253.0,NaN,9752242.0,510723.0,54872.0,-2921924.0
542081,901680258,2023,YARUMAL-ANTIOQUIA,C1040 - Elaboración de productos lácteos,ANTIOQUIA,23399409.0,12826402.0,8078725.0,34960432.0,3296413.0,...,144877391.0,11045290.0,1042317,4429261.0,3534786.0,4649947.0,2007031.0,546904.0,NaN,20430859.0
542082,901712459,2023,BOGOTA D.C.,H4921 - Transporte de pasajeros,BOGOTA D.C.,16694054.0,NaN,310128098.0,326822152.0,-3508793.0,...,25423333.0,11101483.0,-4083819,12593401.0,16818429.0,0.0,4400572.0,0.0,14321850.0,332940751.0


✅ ¡Proceso completado!
📦 Registros únicos por NIT y Año: 541714

✅ `2_BaseInicial.pkl` guardado en: /content/drive/MyDrive/Datos/2_1_BaseInicial.pkl
✅ `2_BaseInicial.parquet` guardado en: /content/drive/MyDrive/Datos/2_2_BaseInicial.parquet

📊 RESUMEN DE LA BASE FINAL
✅ Total de registros únicos: 541714
👤 NITs únicos: 69423
📅 Años únicos: 29
📄 Total de columnas: 26

🆔 Ejemplos de NITs:
['860067934' '890328003' '891302216' '800160353' '800063990' '891300003'
 '890500936' '800054093' '860528519' '800145912']

📅 Años presentes:
[1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004]

📋 Columnas disponibles:
   • NIT
   • Año
   • Ciudad
   • CIIU
   • DEP
   • AC
   • I
   • AF
   • AT
   • PN
   • PLP
   • PCP
   • PT
   • CPC
   • CPP
   • UN
   • V
   • UB
   • EBIT
   • UO
   • GF
   • GV
   • GA
   • Imp
   • DyA
   • FCL
📝 Resumen guardado en: /content/drive/MyDrive/Datos/2_resumen.txt
